# Dataframe for Toronto Neighborhoods

Import the libraries we need.

In [49]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011


In [74]:
!pip install folium

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


In [85]:
!pip install geopy
from geopy.geocoders import Nominatim 

In [142]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 

Scrape the content from the wikipedia website. 

In [143]:
html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=urllib.request.urlopen(html).read()
soup=BeautifulSoup(source, 'html.parser')

Make the html content into a table.

In [144]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = df[0]

Remove not assigned values.

In [145]:
df = df[df["Borough"] != "Not assigned"]

Merge those values that have the same Postal Code.

In [146]:
df = df.groupby(['Postal Code', "Borough"])["Neighborhood"].apply(lambda x: ','.join(x)).reset_index()
df.shape

(103, 3)

In [147]:
import geocoder 

The geocoder is not working, but I am pretty sure my code is fine. There is no error message, the code just keeps running forever....

In [81]:
# initialize your variable to None
lat_lng_coords = None
latitude = dict()
longitude = dict()

for postal_code in df["Postal Code"]:
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.update(postal code:lat_lng_coords[0])
    longitude.ipdate(postal code:lat_lng_coords[1])


SyntaxError: invalid syntax (<ipython-input-81-bf26b042b08b>, line 12)

So I add the data manually from the csv.

In [148]:
df = df.merge(pd.read_csv("http://cocl.us/Geospatial_data"),"left", left_on = "Postal Code", right_on = "Postal Code")

In [149]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Now, let's explore the neighborhoods.

Frst, I find the location for Toronto.

In [150]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Then I map Toronto and add all the neighborhoods

In [151]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We can see that the neighborhoods cluster tighter together at the coastline, what seems to be the center of Toronto and then spread out from there.

In [152]:
# The code was removed by Watson Studio for sharing.

Now, let's check out some neighborhoods. Let's take the first neighborhood: Malvern, Rouge. This can be done with any of them.

In [153]:
df.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [154]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [155]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [156]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
import requests
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items)

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(15)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Downtown Toronto,Neighborhood,NaN,CA,Toronto,Canada,NaN,113,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,5227bb01498e17bf485e6202
1,Nathan Phillips Square,Plaza,100 Queen St W,CA,Toronto,Canada,at Bay St,138,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65227047322295...",43.652270,-79.383516,NaN,M5H 2N1,ON,4ad4c05ef964a520a6f620e3
2,Chatime 日出茶太,Bubble Tea Shop,132 Dundas St W,CA,Toronto,Canada,btwn Bay & University,237,"[132 Dundas St W (btwn Bay & University), Toro...","[{'label': 'display', 'lat': 43.65554164147378...",43.655542,-79.384684,NaN,M5G 1C3,ON,4e2284b11fc7c0ef9857d143
3,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,230,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
4,Indigo,Bookstore,220 Yonge St,CA,Toronto,Canada,NaN,260,"[220 Yonge St, Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65351471121164...",43.653515,-79.380696,Downtown Yonge,M5B 2H1,ON,4b2a6eb8f964a52012a924e3
5,Japango,Sushi Restaurant,122 Elizabeth St.,CA,Toronto,Canada,at Dundas St. W,222,"[122 Elizabeth St. (at Dundas St. W), Toronto ...","[{'label': 'display', 'lat': 43.65526771691681...",43.655268,-79.385165,NaN,M5G 1P5,ON,4ae7b27df964a52068ad21e3
6,LUSH,Cosmetics Shop,"220 Yonge St, Unit B215-A",CA,Toronto,Canada,in Toronto Eaton Centre,284,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","[{'label': 'display', 'lat': 43.653557, 'lng':...",43.653557,-79.380400,NaN,M5B 2H1,ON,4bd0b30d41b9ef3b8fa0fae5
7,Poke Guys,Poke Place,112 Elizabeth St,CA,Toronto,Canada,at Dundas St W,181,"[112 Elizabeth St (at Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.65489527525682...",43.654895,-79.385052,NaN,M5G 1P5,ON,57bcd3b7498e652a678d0378
8,Old City Hall,Monument / Landmark,60 Queen Street West,CA,Toronto,Canada,NaN,240,"[60 Queen Street West, Toronto ON M5H 1A1, Can...","[{'label': 'display', 'lat': 43.65200880087612...",43.652009,-79.381744,NaN,M5H 1A1,ON,4ad4c05ef964a5208ef620e3
9,M Square Coffee Co,Coffee Shop,C24 - 123 Queen St W,CA,Toronto,Canada,NaN,253,"[C24 - 123 Queen St W, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65121797253777...",43.651218,-79.383555,NaN,NaN,ON,54132b3b498ee9ca9332e189


We can see there is quite the selection of venues in this neighborhood.
Let's check for Hotels.

In [157]:
dataframe_filtered[dataframe_filtered["categories"] == "Hotel"]

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
12,DoubleTree by Hilton,Hotel,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,204,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,52ce14b0498e50457ce11780


There appears to be one hotel in this neighborhood. Let's map it.

In [158]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add hotel spot to the map as blue circle markers
lat = dataframe_filtered.loc[12, "lat"]
lng = dataframe_filtered.loc[12, "lng"]
label = dataframe_filtered.loc[12, "name"]
folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    fill=True,
    color='blue',
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Cool, it seems to be right at the center of our neighborhood. Let's see if there's any cool things to do around the hotel. I want to go shopping and to a museum. Let's check it out. 

In [159]:
df_shopping = dataframe_filtered.loc[dataframe_filtered["categories"].isin(["Shopping Center", "Art Museum", "Clothing Store"])]
df_shopping.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
3,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,230,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
17,UNIQLO ユニクロ,Clothing Store,220 Yonge St,CA,Toronto,Canada,at Dundas St W,378,"[220 Yonge St (at Dundas St W), Toronto ON M5B...","[{'label': 'display', 'lat': 43.65591027779457...",43.655910,-79.380641,Downtown Toronto,M5B 2H1,ON,57eda381498ebe0e6ef40972
19,Nordstrom,Clothing Store,260 Yonge Street,CA,Toronto,Canada,NaN,295,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]","[{'label': 'display', 'lat': 43.655041, 'lng':...",43.655041,-79.380966,NaN,M5B 2L9,ON,57d1e1a2498e96cce6f32217


In [160]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add hotel spots to the map as blue circle markers
lat = dataframe_filtered.loc[12, "lat"]
lng = dataframe_filtered.loc[12, "lng"]
label = dataframe_filtered.loc[12, "name"]
folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    fill=True,
    color='blue',
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(venues_map)

# add shopping spots to the map as blue circle markers
for lat, lng, label in zip(df_shopping.lat, df_shopping.lng, df_shopping.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

Perfect, looks like the museum is really close!

Let's see if these venues have good ratings.

In [161]:
for venue_id in df_shopping["id"]:

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

8.2
8.7
7.9


yay, they all look pretty ok.